In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import glob

df = pd.read_csv("sampled_segmentation_reviews.csv")
print(f"Total data: {len(df)} baris")

Couldn't import dot_parser, loading of dot files will not be possible.
Total data: 1866 baris


In [ ]:
# Ganti NaN jadi string kosong, lalu hitung panjangnya
df['char_count'] = df['review'].fillna("").apply(len)

# Hitung rata-rata
average_chars = df['char_count'].mean()

print(f"Rata-rata karakter per review: {average_chars:.2f}")


Rata-rata karakter per Review: 330.27


In [6]:
# 2. Pastikan kolom 'reviews' ada dan tidak berisi NaN
df = df.dropna(subset=["review"])

# 3. Ambil 10 sampel acak
sample_reviews = df["review"].sample(n=10, random_state=42).reset_index(drop=True)

# 4. Tampilkan hasil
print(sample_reviews)

0    Ph'nglui mglw'nafh Cthulhu R'lyeh wgah'nagl fh...
1                       WAY better than Halo Infinite.
2    Amazing story and amazing gameplay but the mai...
3    Pikuniku is a short, but pretty good and charm...
4    Batman villains are interesting, Batman himsel...
5                                              Amazing
6    I have seriously no idea why it's such a buggy...
7    Easily the best computer card/board game I've ...
8                                            Fun game.
9    Fun & Addictive This is a really easy and nice...
Name: review, dtype: object


In [7]:
import ssl
import nltk

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Sekarang bisa download
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /Users/divaoncom/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/divaoncom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Install hanya sekali
!pip install nltk spacy
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 296.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 634.4/634.4 kB 282.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.9/844.9 kB 319.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 292.5 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 280.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 278.1 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 

In [8]:
import pandas as pd
import re
import nltk
import spacy

# Load data
df = pd.read_csv("sampled_segmentation_reviews.csv")
df['review'] = df['review'].astype(str)

# Tampilkan jumlah data
print("Jumlah data:", len(df))
df[['review']].head()


Jumlah data: 1866


,review
0,I like to push npcs into cars so they die
1,damn solving a case is very satisfying
2,Play it like detective TV series. 1 case per day.
3,some cases are really easy af but some cases a...
4,"Playing as the police, the other side of the G..."


In [9]:
print("Jumlah data sebelum Case Folding:", df['review'].notnull().sum())

df['case_folding'] = df['review'].str.lower()

print("Jumlah data sesudah Case Folding:", df['case_folding'].notnull().sum())
print("Contoh:\nSebelum:", df['review'][0], "\nSesudah:", df['case_folding'][0])


Jumlah data sebelum Case Folding: 1866
Jumlah data sesudah Case Folding: 1866
Contoh:
Sebelum: I like to push npcs into cars so they die 
Sesudah: i like to push npcs into cars so they die


In [10]:
print("Jumlah data sebelum Cleansing:", df['case_folding'].notnull().sum())

df['cleansing'] = df['case_folding'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))

print("Jumlah data sesudah Cleansing:", df['cleansing'].notnull().sum())
print("Contoh:\nSebelum:", df['case_folding'][0], "\nSesudah:", df['cleansing'][0])


Jumlah data sebelum Cleansing: 1866
Jumlah data sesudah Cleansing: 1866
Contoh:
Sebelum: i like to push npcs into cars so they die 
Sesudah: i like to push npcs into cars so they die


In [11]:
normalization_dict = {
    "u": "you", "r": "are", "ur": "your", "pls": "please", "plz": "please",
    "thx": "thanks", "ty": "thank you", "btw": "by the way", "lol": "laughing out loud",
    "lmao": "laughing my ass off", "omg": "oh my god", "wtf": "what the fuck",
    "idk": "i do not know", "imo": "in my opinion", "imho": "in my humble opinion",
    "afaik": "as far as i know", "brb": "be right back", "bbl": "be back later",
    "gg": "good game", "ez": "easy", "nerf": "weaken", "buff": "strengthen",
    "noob": "newbie", "grind": "repetitive task", "op": "overpowered",
    "laggy": "slow connection", "gitgud": "get good", "f2p": "free to play",
    "p2w": "pay to win", "dlc": "downloadable content", "npc": "non player character",
    "fps": "frames per second", "afk": "away from keyboard", "xp": "experience points",
    "lvl": "level", "bossfight": "boss fight", "rng": "random number generator",
    "camping": "staying in one spot", "gank": "ambush attack", "re": "regarding",
    "af": "as fuck", "xd": "bad", "f": "fuck", "ive": "i have", "rp": "roleplay",
    "fckin": "fucking", "tl": "too long", "dr": "didnt read", "lcg": "living card game", 
    "stori": "story", "differ": "different"
}

def normalize(text):
    return ' '.join([normalization_dict.get(word, word) for word in text.split()])

print("Jumlah data sebelum Normalization:", df['cleansing'].notnull().sum())

df['normalization'] = df['cleansing'].apply(normalize)

print("Jumlah data sesudah Normalization:", df['normalization'].notnull().sum())
print("Contoh:\nSebelum:", df['cleansing'][0], "\nSesudah:", df['normalization'][0])


Jumlah data sebelum Normalization: 1866
Jumlah data sesudah Normalization: 1866
Contoh:
Sebelum: i like to push npcs into cars so they die 
Sesudah: i like to push npcs into cars so they die


In [12]:
def tokenize(text):
    return text.split()

print("Jumlah data sebelum Tokenizing:", df['normalization'].notnull().sum())

df['tokenized'] = df['normalization'].apply(tokenize)

print("Jumlah data sesudah Tokenizing:", df['tokenized'].notnull().sum())
print("Contoh:\nSebelum:", df['normalization'][0], "\nSesudah:", df['tokenized'][0])


Jumlah data sebelum Tokenizing: 1866
Jumlah data sesudah Tokenizing: 1866
Contoh:
Sebelum: i like to push npcs into cars so they die 
Sesudah: ['i', 'like', 'to', 'push', 'npcs', 'into', 'cars', 'so', 'they', 'die']


In [14]:
pip install spacy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 301.9 kB/s eta 0:00:0000:0100:02
  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 330.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.0/779.0 kB 443.6 kB/s eta 0:00:00a 0:00:01
Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 255.9 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
  Attempting uninstall: blis
    Found existing installation: blis 1.3.0
    Uninstalling blis-1.3.0:
      Successfully uninstalled blis-1.3.0
  Attempting uninstall: thinc
    Found existing installation: thinc 8.3.6
    Uninstalling thinc-8.3.6:
      Successfully un

In [33]:
pip install spacy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [34]:
!python -m spacy download en_core_web_sm

zsh:1: command not found: python


In [13]:
nlp = spacy.load("en_core_web_sm")

def lemmatize(tokens):
    doc = nlp(' '.join(tokens))
    return [token.lemma_ for token in doc]

print("Jumlah data sebelum Lemmatization:", df['tokenized'].notnull().sum())

df['lemmatized'] = df['tokenized'].apply(lemmatize)

print("Jumlah data sesudah Lemmatization:", df['lemmatized'].notnull().sum())
print("Contoh:\nSebelum:", df['tokenized'][0], "\nSesudah:", df['lemmatized'][0])


Jumlah data sebelum Lemmatization: 1866
Jumlah data sesudah Lemmatization: 1866
Contoh:
Sebelum: ['i', 'like', 'to', 'push', 'npcs', 'into', 'cars', 'so', 'they', 'die'] 
Sesudah: ['I', 'like', 'to', 'push', 'npc', 'into', 'car', 'so', 'they', 'die']


In [14]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

print("Jumlah data sebelum Stopword Removal:", df['lemmatized'].notnull().sum())

df['final_tokens'] = df['lemmatized'].apply(remove_stopwords)

print("Jumlah data sesudah Stopword Removal:", df['final_tokens'].notnull().sum())
print("Contoh:\nSebelum:", df['lemmatized'][0], "\nSesudah:", df['final_tokens'][0])


Jumlah data sebelum Stopword Removal: 1866
Jumlah data sesudah Stopword Removal: 1866
Contoh:
Sebelum: ['I', 'like', 'to', 'push', 'npc', 'into', 'car', 'so', 'they', 'die'] 
Sesudah: ['I', 'like', 'push', 'npc', 'car', 'die']


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/divaoncom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
import pandas as pd

# Asumsi df sudah dimuat dari file CSV sebelumnya

# Gabungkan hasil akhir ke dalam kolom teks baru (cleaned_reviews)
df['cleaned_reviews'] = df['final_tokens'].apply(lambda x: ' '.join(x))

# Pilih dan urutkan kolom yang akan disimpan
output_columns = [
    'game',               # nama game (pastikan kolom ini ada di file asli)
    'review',             # review asli
    'case_folding',
    'cleansing',
    'normalization',
    'tokenized',
    'lemmatized',
    'final_tokens',       # hasil setelah stopword removal
    'cleaned_reviews'     # versi string final
]

# Periksa apakah kolom 'game' ada, jika tidak, beri peringatan
if 'game' not in df.columns:
    print("⚠️ Kolom 'game' tidak ditemukan! Pastikan ada di file CSV asli.")
    if 'game' in output_columns:
        output_columns.remove('game')

# Simpan ke CSV
df[output_columns].to_csv("preprocessed_reviews.csv", index=False)

print("✅ File berhasil disimpan sebagai 'preprocessed_reviews.csv'")


✅ File berhasil disimpan sebagai 'preprocessed_reviews.csv'


In [17]:
df = pd.read_csv("preprocessed_reviews.csv")
df.head(10)

,game,review,case_folding,cleansing,normalization,tokenized,lemmatized,final_tokens,cleaned_reviews
0,L.A. Noire,I like to push npcs into cars so they die,i like to push npcs into cars so they die,i like to push npcs into cars so they die,i like to push npcs into cars so they die,"['i', 'like', 'to', 'push', 'npcs', 'into', 'c...","['I', 'like', 'to', 'push', 'npc', 'into', 'ca...","['I', 'like', 'push', 'npc', 'car', 'die']",I like push npc car die
1,L.A. Noire,damn solving a case is very satisfying,damn solving a case is very satisfying,damn solving a case is very satisfying,damn solving a case is very satisfying,"['damn', 'solving', 'a', 'case', 'is', 'very',...","['damn', 'solve', 'a', 'case', 'be', 'very', '...","['damn', 'solve', 'case', 'satisfying']",damn solve case satisfying
2,L.A. Noire,Play it like detective TV series. 1 case per day.,play it like detective tv series. 1 case per day.,play it like detective tv series case per day,play it like detective tv series case per day,"['play', 'it', 'like', 'detective', 'tv', 'ser...","['play', 'it', 'like', 'detective', 'tv', 'ser...","['play', 'like', 'detective', 'tv', 'series', ...",play like detective tv series case per day
3,L.A. Noire,some cases are really easy af but some cases a...,some cases are really easy af but some cases a...,some cases are really easy af but some cases a...,some cases are really easy as fuck but some ca...,"['some', 'cases', 'are', 'really', 'easy', 'as...","['some', 'case', 'be', 'really', 'easy', 'as',...","['case', 'really', 'easy', 'fuck', 'case', 'ha...",case really easy fuck case hard beat final bos...
4,L.A. Noire,"Playing as the police, the other side of the G...","playing as the police, the other side of the g...",playing as the police the other side of the gt...,playing as the police the other side of the gt...,"['playing', 'as', 'the', 'police', 'the', 'oth...","['play', 'as', 'the', 'police', 'the', 'other'...","['play', 'police', 'side', 'gta', 'coin', 'dea...",play police side gta coin deal mess maniacs le...
5,L.A. Noire,HIGS REPRESENT,higs represent,higs represent,higs represent,"['higs', 'represent']","['hig', 'represent']","['hig', 'represent']",hig represent
6,L.A. Noire,"Long time ago, PC gamers killed the Mother of ...","long time ago, pc gamers killed the mother of ...",long time ago pc gamers killed the mother of r...,long time ago pc gamers killed the mother of r...,"['long', 'time', 'ago', 'pc', 'gamers', 'kille...","['long', 'time', 'ago', 'pc', 'gamer', 'kill',...","['long', 'time', 'ago', 'pc', 'gamer', 'kill',...",long time ago pc gamer kill mother rockstar ce...
7,L.A. Noire,Very good game 12/10,very good game 12/10,very good game,very good game,"['very', 'good', 'game']","['very', 'good', 'game']","['good', 'game']",good game
8,L.A. Noire,"The movie L.A Confidential, except it's a game...","the movie l.a confidential, except it's a game...",the movie la confidential except its a game be...,the movie la confidential except its a game be...,"['the', 'movie', 'la', 'confidential', 'except...","['the', 'movie', 'la', 'confidential', 'except...","['movie', 'la', 'confidential', 'except', 'gam...",movie la confidential except game well play bl...
9,L.A. Noire,"amazing game, it's a nice change to be on the ...","amazing game, it's a nice change to be on the ...",amazing game its a nice change to be on the po...,amazing game its a nice change to be on the po...,"['amazing', 'game', 'its', 'a', 'nice', 'chang...","['amazing', 'game', 'its', 'a', 'nice', 'chang...","['amazing', 'game', 'nice', 'change', 'police'...",amazing game nice change police side mayhem ca...
